In [1]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
import string
import os
from PIL import Image
import glob
import pickle
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, Flatten, Reshape, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
# load descriptions
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

def load_descriptions(doc):  
    mapping = dict()
    for line in doc.split('\n'):
        tokens = line.split()
        if len(line) < 2:
            continue
        image_id, image_desc = tokens[0], tokens[1:]
        image_id = image_id.split('.')[0]
        image_desc = ' '.join(image_desc)
        if image_id not in mapping:
            mapping[image_id] = list()
        mapping[image_id].append(image_desc)
    return mapping
  
def clean_descriptions(descriptions):
    table = str.maketrans('', '', string.punctuation)
    for key, desc_list in descriptions.items():
        for i in range(len(desc_list)):
            desc = desc_list[i]
            desc = desc.split()
            desc = [word.lower() for word in desc]
            desc = [w.translate(table) for w in desc]
            desc = [word for word in desc if len(word)>1]
            desc = [word for word in desc if word.isalpha()]
            desc =  ' '.join(desc)
            desc = desc.replace("merrygoround", "apples")
            desc = desc.replace("tshirt", "t-shirt")
            desc = desc.replace("sweat-shirt", "sweatshirt")
            desc = desc.replace("upsidedown", "upside-down")
            desc = desc.replace("snowcovered", "snow-covered")
            desc = desc.replace("rollerblader", "person")
            desc = desc.replace("waterskier", "person")
            desc = desc.replace("floaties", "floaty")
            desc_list[i] = desc
    return descriptions

# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc)
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()


# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
    doc = load_doc(filename)
    descriptions = dict()
    for line in doc.split('\n'):
        tokens = line.split()
        image_id, image_desc = tokens[0], tokens[1:]
        if image_id in dataset:
            if image_id not in descriptions:
                descriptions[image_id] = list()
            desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
            descriptions[image_id].append(desc)
    return descriptions
  
def load_set(filename):
    doc = load_doc(filename)
    dataset = list()
    for line in doc.split('\n'):
        if len(line) < 1:
            continue
        identifier = line.split('.')[0]
        dataset.append(identifier)
    return set(dataset)

# load training dataset 


filename = "dataset/Flickr8k_text/Flickr8k.token.txt"
doc = load_doc(filename)
descriptions = load_descriptions(doc)
descriptions = clean_descriptions(descriptions)
save_descriptions(descriptions, 'descriptions.txt')
filename = 'dataset/Flickr8k_text/Flickr_8k.trainImages.txt'
train = load_set(filename)
train_descriptions = load_clean_descriptions('descriptions.txt', train)
filename = 'dataset/Flickr8k_text/Flickr_8k.devImages.txt'
val = load_set(filename)
val_descriptions = load_clean_descriptions('descriptions.txt', val)

In [3]:
# Create a list of all the training captions
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
        
        
# Consider only words which occur at least 10 times in the corpus
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('Preprocessed words {} -> {}'.format(len(word_counts), len(vocab)))


ixtoword = {}
wordtoix = {}

ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1
    
vocab_size = len(ixtoword) + 1 # one for appended 0's

# Load Glove vectors
glove_dir = 'glove.6B'
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_dim = 200

# Get 200-dim dense vector for each of the words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoix.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Preprocessed words 7575 -> 1649


In [4]:
#список слов, которых нет в GloVe
for word, _ in wordtoix.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is None:
        print(word, word_counts[word])

startseq 30000
endseq 30000


In [5]:
all_images_path = 'dataset/Flicker8k_Dataset/'
# Create a list of all image names in the directory
all_images = glob.glob(all_images_path + '*.jpg')

# Create a list of all the training and testing images with their full path names
def create_list_of_images(file_path):
    images_names = set(open(file_path, 'r').read().strip().split('\n'))
    images = []

    for image in all_images: 
        if image[len(all_images_path):] in images_names:
            images.append(image)

    return images


train_images_path = 'dataset/Flickr8k_text/Flickr_8k.trainImages.txt'
test_images_path = 'dataset/Flickr8k_text/Flickr_8k.testImages.txt'
val_images_path = 'dataset/Flickr8k_text/Flickr_8k.devImages.txt'

train_images = create_list_of_images(train_images_path)
test_images = create_list_of_images(test_images_path)
val_images = create_list_of_images(val_images_path)

#preprocessing the images
def preprocess(image_path):
    img = image.load_img(image_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

# Load the inception v3 model
model = InceptionV3(weights='imagenet')

# Create a new model, by removing the last layer (output layer) from the inception v3
model_new = Model(model.input, model.layers[-2].output)

# Encoding a given image into a vector of size (2048, )
def encode(image):
    image = preprocess(image) 
    fea_vec = model_new.predict(image) 
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1])
    return fea_vec

encoding_train = {}
for img in train_images:
    encoding_train[img[len(all_images_path):]] = encode(img)


encoding_test = {}
for img in test_images:
    encoding_test[img[len(all_images_path):]] = encode(img)

encoding_val = {}
for img in val_images:
    encoding_val[img[len(all_images_path):]] = encode(img)

# Save the bottleneck features to disk
with open("encoded_files/encoded_train_images.pkl", "wb") as encoded_pickle:
    pickle.dump(encoding_train, encoded_pickle)

with open("encoded_files/encoded_test_images.pkl", "wb") as encoded_pickle:
    pickle.dump(encoding_test, encoded_pickle)


with open("encoded_files/encoded_val_images.pkl", "wb") as encoded_pickle:
    pickle.dump(encoding_val, encoded_pickle)    

train_features = pickle.load(open("encoded_files/encoded_train_images.pkl", "rb"))

In [6]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0

In [7]:
import keras.callbacks
max_length1 = 34
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length1,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

model.compile(loss='categorical_crossentropy', optimizer='adam')

epochs = 20
number_pics_per_batch = 3
steps = len(train_descriptions)//number_pics_per_batch

checkpoint = keras.callbacks.ModelCheckpoint('checkp_save/model{epoch:08d}.h5', save_freq=2000) 
generator = data_generator(train_descriptions, train_features, wordtoix, max_length1, number_pics_per_batch)
history = model.fit(generator, epochs=20,  steps_per_epoch=steps, verbose=1, callbacks=[checkpoint])


model.optimizer.lr = 0.0001
epochs = 10
number_pics_per_batch = 6
steps = len(train_descriptions)//number_pics_per_batch

checkpoint = keras.callbacks.ModelCheckpoint('checkp_save/model2{epoch:08d}.h5', save_freq=1000) 

generator = data_generator(train_descriptions, train_features, wordtoix, max_length1, number_pics_per_batch)
history1 = model.fit(generator, epochs=10, steps_per_epoch=steps, verbose=1, callbacks=[checkpoint])
model.save('saved_model/modelswap_' + '34_fit256' + '.h5')

Epoch 1/20
2000/2000 [==============================] - 268s 133ms/step - loss: 4.1176
Epoch 2/20
2000/2000 [==============================] - 260s 130ms/step - loss: 3.4170
Epoch 3/20
2000/2000 [==============================] - 257s 129ms/step - loss: 3.2009
Epoch 4/20
2000/2000 [==============================] - 256s 128ms/step - loss: 3.0669
Epoch 5/20
2000/2000 [==============================] - 278s 139ms/step - loss: 2.9729
Epoch 6/20
2000/2000 [==============================] - 293s 146ms/step - loss: 2.9012
Epoch 7/20
2000/2000 [==============================] - 286s 143ms/step - loss: 2.8446
Epoch 8/20
2000/2000 [==============================] - 260s 130ms/step - loss: 2.7946
Epoch 9/20
2000/2000 [==============================] - 259s 130ms/step - loss: 2.7523
Epoch 10/20
2000/2000 [==============================] - 260s 130ms/step - loss: 2.7209
Epoch 11/20
2000/2000 [==============================] - 257s 129ms/step - loss: 2.6908
Epoch 12/20
2000/2000 [==================